Bad News merges three datasources to create a map of the united states to overlap with police shooting information provided by the washingtonpost on github.

    https://github.com/washingtonpost/data-police-shootings
    https://gist.githubusercontent.com/mshafrir/2646763/raw/8b0dbb93521f5d6889502305335104218454c2bf/states_hash.json
    

In [8]:
    from articles import *; from pandas import *; from bokeh import plotting, models, resources
    plotting.output_notebook(resources=resources.CDN)

Loading BokehJS ...

In [2]:
    path = the.Path('states.csv')()
    if not path.exists():
        io.json.json_normalize(
            a.request('get')[this().json()].get('features')("https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json")
                ).to_csv(path)
    states = read_csv(path)
    states['geometry.coordinates'] = states['geometry.coordinates'].apply(__import__('ast').literal_eval)

    pluck = lambda ___: a.map(a.pluck(___).list()).list()[Series]

    poly = states[states['geometry.type'].eq('Polygon')]
    poly = concat({
        'x': poly['geometry.coordinates'].apply(pluck(0)),
        'y': poly['geometry.coordinates'].apply(pluck(1)),
    }, axis=1).stack(-1).reset_index(-1, drop=True).join(poly)

In [3]:
    multi = states[states['geometry.type'].eq('MultiPolygon')]
    multi = concat({
        'x': multi['geometry.coordinates'].apply(Series).stack().apply(pluck(0))[0],
        'y': multi['geometry.coordinates'].apply(Series).stack().apply(pluck(1))[0]
    }, axis=1).reset_index(-1, drop=True).join(multi)
    
    states = concat([poly, multi]).set_index('properties.name')

In [4]:
    path = the.Path('massshooting.csv')()
    if not path.exists():
        read_csv("https://raw.githubusercontent.com/washingtonpost/data-police-shootings/master/fatal-police-shootings-data.csv"
                ).to_csv(path)
    df = read_csv(path)
    df = df.set_index(df.date.pipe(to_datetime))

In [5]:
    path = the.Path('abbrstate.json')()
    if not path.exists():(
        a.request('get')[this().json()].dumps()[path.write_text]
    )("https://gist.githubusercontent.com/mshafrir/2646763/raw/8b0dbb93521f5d6889502305335104218454c2bf/states_hash.json")
    
    abbr = the.loads()(path.read_text())

In [6]:
    positions = states[[*'xy']].applymap(np.mean).groupby(states.index).mean()#.pipe(plotting.ColumnDataSource)

    positions = df.state.value_counts().to_frame('incidents').pipe(
        lambda df: df.set_index(df.index.map(abbr.__getitem__))
    ).join(positions).dropna()

    from sklearn.preprocessing import *

    positions = positions.incidents.to_frame('scale').astype(float).apply(a(feature_range=(10, 50))[minmax_scale]).applymap(
        "{}pt".format).join(positions)

    positions[['incidents']].T

,California,Texas,Florida,Arizona,Ohio,Colorado,Oklahoma,Georgia,North Carolina,Missouri,...,Montana,Hawaii,Connecticut,South Dakota,Delaware,Wyoming,New Hampshire,North Dakota,Vermont,Rhode Island
incidents,463,240,173,130,88,82,81,79,76,71,...,12,11,11,10,8,8,7,5,4,2


In [7]:
    p = plotting.figure()
    source = plotting.ColumnDataSource(states)
    p.text('x', 'y', 'incidents', text_font_size="scale", source=positions)
    p.text('x', 'y', 'index', text_font_size='13pt', source=positions)
    p.patches('x', 'y', fill_alpha=0, source=source)
    show = lambda:plotting.show(p)